In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
pip install datasets evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, pipeline, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load as load_metric
import logging
import os
import re
from PIL import Image
import datasets
import io

In [ ]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
MODELS = {
    "UI-TARS 7B SFT": "bytedance-research/UI-TARS-7B-DPO"
}

In [ ]:
def load_model(model_path):
    """Load model and processor with Hugging Face authentication."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    hf_token = os.getenv("HUGGINGFACE_TOKEN")

    """Load model and processor."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.cuda.empty_cache()
    model = Qwen2VLForConditionalGeneration.from_pretrained(

        model_path,
        torch_dtype=torch.float16,
        device_map={"": 0},
        offload_folder="offload",
        quantization_config=BitsAndBytesConfig(load_in_8bit=True),
        token=hf_token
    )
    processor = AutoProcessor.from_pretrained(model_path, token=hf_token)
    logger.info(f"{model_path} Model Loaded Successfully!")
    return model, processor

In [ ]:
def evaluate_vqa(model, processor, dataset_name="HuggingFaceM4/VQAv2"):
    """Evaluate on VQAv2 dataset for Visual Question Answering."""
    dataset = load_dataset(dataset_name, split="validation[:5]", trust_remote_code=True)
    metric = load_metric("squad_v2")  # Using SQuAD metric for VQA

    results = []

    for example in dataset:
        image_data = example["image"]

        # Handle different formats
        if isinstance(image_data, bytes):
            image = Image.open(io.BytesIO(image_data)).convert("RGB")  # Convert bytes to image
        elif isinstance(image_data, dict) and "path" in image_data:
            image = Image.open(image_data["path"]).convert("RGB")  # Load from path
        elif isinstance(image_data, str):
            image = Image.open(image_data).convert("RGB")  # Direct path case
        else:
            raise ValueError(f"Unexpected image format: {type(image_data)}")  # Debugging

        question = example["question"]

        # Process image and question
        inputs = processor(images=image, text=question, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
        outputs = model.generate(**inputs, max_new_tokens=50)
        predicted_answer = processor.batch_decode(outputs, skip_special_tokens=True)[0]

        # Store results
        results.append({
            "image": example["image_id"],  # Image ID for reference
            "question": question,
            "ground_truth": example["answers"]["text"][0],  # Take the first ground truth answer
            "predicted": predicted_answer
        })

        # Compute metric
        metric.add(
            predictions={"id": example["question_id"], "prediction_text": predicted_answer},
            references={"id": example["question_id"], "answers": example["answers"]}
        )

    # Print results
    for res in results:
        print(f"\n🖼 Image ID: {res['image']}")
        print(f"❓ Question: {res['question']}")
        print(f"✅ Ground Truth: {res['ground_truth']}")
        print(f"🤖 Model Prediction: {res['predicted']}")

    return metric.compute()

In [ ]:
for model_name, model_path in MODELS.items():
    print(f"\nEvaluating {model_name}...")
    model, processor = load_model(model_path)

    print("Squad QA:", evaluate_vqa(model, processor))


Evaluating UI-TARS 7B SFT...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/352 [00:00<?, ?B/s]

VQAv2.py:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


FSTimeoutError: 